In [ ]:
import socket
import json
import logging
import requests
import traceback

# Setup logging
logging.basicConfig(level=logging.INFO)
## low mesh mawgoda no data printed

# Flask app URL (to send data to)
FLASK_URL = "http://127.0.0.1:8000/update-tank"
#The FLASK_URL variable is the endpoint where data will be sent (a Flask app listening on port 8000).
# Create a socket server
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
#A socket is created with IPv4 addressing (AF_INET) and TCP/IP protocol (SOCK_STREAM).
server.bind(("0.0.0.0", 12345))
server.listen(5)
#The server is bound to listen on all interfaces (0.0.0.0) at port 12345. The listen(5) call sets up a backlog queue of 5 connections.
logging.info("Server is listening on port 12345")

try:
    while True:
        #The server.accept() function blocks and waits for a client connection. When a client connects, a new socket object (client) and the address of the client (addr) are returned.
        client, addr = server.accept()
        logging.info(f"Connection established with {addr}")
        try:
            data = client.recv(1024)
            #The server waits for data from the client by calling recv(1024) (reads up to 1024 bytes).
            if data:
                logging.debug(f"Raw data received: {data}")
                #If data is received, it is logged at the DEBUG level and decoded from bytes to a string.
                try:
                    # Parse JSON data
                    message = json.loads(data.decode())
                    #The json.loads() method is used to convert the string into a Python dictionary (JSON). If successful, the parsed message is logged and printed.
                    logging.info(f"Received: {message}")
                    print(message)
                    # Send the data to Flask app
                    response = requests.post(FLASK_URL, json=message)
                    #The parsed data (message) is sent to the Flask app using the requests.post() method, with the URL specified by FLASK_URL. The data is sent as a JSON payload (json=message).
                    if response.status_code == 200: #ok
                        logging.info("Data successfully sent to Flask app.")
                    else:
                        logging.error(f"Failed to send data to Flask app. Status code: {response.status_code}")
                except json.JSONDecodeError as e:
                    logging.error(f"JSON error: {e}, raw data: {data}")
            else:
                logging.warning("No data received from the client.")
        except Exception as e:
            logging.error(f"Unexpected error: {traceback.format_exc()}")
        finally:
            client.close()
except KeyboardInterrupt:
    logging.info("Shutting down server...")
finally:
    server.close()


INFO:root:Server is listening on port 12345
